In [2]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the selected model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA A10. Max memory: 21.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [3]:
import pandas as pd
from pprint import pprint

list_ds =[
{"instruction":"Which company is the Acquirer.", "input":"Peak Performance Corporation to Acquire Power Forward Corporation by 28/Jul/2022", "output":"Peak Performance Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"Prime Solutions Group Acquires Dynamic Solutions Inc in a Strategic Merger", "output":"Prime Solutions Group"},
{"instruction":"Which company is the Acquirer.", "input":"Wonderful Products Inc, a leading company in the consumer goods industry, has successfully completed its acquisition of Noble Solutions Group, a prominent provider of innovative solutions.", "output":"Wonderful Products Inc"},

{"instruction":"Which company is the Acquirer.", "input":"ABC Corporation has completed acquisition of 91% stake in French IT Consulting Company, Rene Consulting", "output":"ABC Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"JBL Technologies has completed acquisition of 51% stake in California Based Defence Company, MaryLand", "output":"JBL Technologies"},

 {"instruction":"Which company is the Acquirer.", "input":"New York, NY - Empire Innovations Inc, a leading technology conglomerate, announced today its plans to acquire Unique Ventures LLC, a prominent venture capital firm specializing in disruptive startups.", "output":"Empire Innovations Inc"},
{"instruction":"Which company is the Acquirer.", "input":"Summit Solutions Inc, a leading technology solutions provider, has announced its plans to acquire Accelerate Innovations Group, a prominent innovation and consulting firm.", "output":"Summit Solutions Inc"}
]



In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

import datasets
import pandas as pd
# Create a dataset from the array of dictionaries

df = pd.DataFrame(list_ds)
dataset = datasets.Dataset.from_pandas(df)

# Print the dataset
print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 7
})


Map: 100%|██████████| 7/7 [00:00<00:00, 2913.58 examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 7
})


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processes to use for data preprocessing
    packing = False, # Whether to pack multiple sequences into one batch to increase training efficiency
    args = TrainingArguments(
        per_device_train_batch_size = 2, #The batch size per device
        gradient_accumulation_steps = 4, #Number of gradient accumulation steps, which allows for effectively larger batch sizes
        warmup_steps = 5, #Number of steps to perform linear learning rate warmup
        max_steps = 60, #Total number of training steps
        learning_rate = 2e-5,#The learning rate for the optimizer
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()


Map (num_proc=2): 100%|██████████| 7/7 [00:00<00:00, 11.35 examples/s]
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.328100
2,3.421900
3,3.421900
4,3.328100
5,3.328100
6,3.296900
7,3.281200
8,3.359400
9,3.281200
10,3.218800


In [12]:
mna_news_input = """HCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022."""

mna_news_instruction1 = """What is the acquistion date. Answer precisely"""
mna_news_instruction2 = """Which company is the Acquirer. . Answer precisely """
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        mna_news_input, # instruction
        mna_news_instruction1, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
generated_text = tokenizer.batch_decode(outputs)
generated_text

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nHCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022.\n\n### Input:\nWhat is the acquistion date. Answer precisely\n\n### Response:\nJanuary 05, 2022<|end_of_text|>']

In [13]:
response = generated_text[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]

In [14]:
response

'January 05, 2022<|end_of_text|>'

In [15]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...")
# tokenizer.push_to_hub("your_name/lora_model", token = "...")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [16]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

In [20]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        mna_news_input, # instruction
        mna_news_instruction2, # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
generated_text = tokenizer.batch_decode(outputs)

In [21]:
response = generated_text[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]

In [22]:
response

'HCL Technologies<|end_of_text|>'